https://zenn.dev/umi_mori/books/prompt-engineer/viewer/langchain_memory

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGCHAIN_TRACING"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "http://localhost:8000"

In [2]:
from langchain.memory import ChatMessageHistory
from langchain.schema import messages_to_dict
from langchain.schema import messages_from_dict

history = ChatMessageHistory()

history.add_user_message("AIとは何？")
history.add_ai_message("AIとは、人工知能のことです。")

message_list = history.messages
message_dict = messages_to_dict(message_list)
print(message_list)
print(message_dict)

message_list_from_dict = messages_from_dict(message_dict)
print(message_list_from_dict)

[HumanMessage(content='AIとは何？', additional_kwargs={}, example=False), AIMessage(content='AIとは、人工知能のことです。', additional_kwargs={}, example=False)]
[{'type': 'human', 'data': {'content': 'AIとは何？', 'additional_kwargs': {}, 'example': False}}, {'type': 'ai', 'data': {'content': 'AIとは、人工知能のことです。', 'additional_kwargs': {}, 'example': False}}]
[HumanMessage(content='AIとは何？', additional_kwargs={}, example=False), AIMessage(content='AIとは、人工知能のことです。', additional_kwargs={}, example=False)]


In [3]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

llm = OpenAI(model_name="text-davinci-003")
prompt_1 = PromptTemplate(
    input_variables=["adjective", "job", "time"],
    template="{time}において、{adjective}{job}に一番オススメのプログラミング言語は?\nプログラミング言語：",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key="programming_language")

prompt_2 = PromptTemplate(
    input_variables=["programming_language", "time"],
    template="現在は{time}だとします。{programming_language}を学ぶためにやるべきことを3ステップで100文字で教えて。",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2, output_key="learning_step")

overall_chain = SequentialChain(
    chains=[chain_1, chain_2],
    input_variables=["adjective", "job"],
    output_variables=["programming_language", "learning_step"],
    verbose=True,
    memory=SimpleMemory(memories={"time": "2030年"}),
)
output = overall_chain({
    "adjective": "ベテランの",
    "job": "エンジニア",
})
print(output)



> Entering new SequentialChain chain...

> Finished chain.
{'adjective': 'ベテランの', 'job': 'エンジニア', 'time': '2030年', 'programming_language': 'Python\n\nPythonは、実践的なプログラミングを可能にするために設計されているオープンソースのプログラミング言語であり、現在では多くのプログラミングを行うために最も人気のある言語の1つとなっています。Pythonは、データサイエンスやAIなどの新しいテクノロジーの分野でも広く利用されています。また、Pythonを使用することで、多くのプラットフォームでソフトウェアを開発することが可能であり、より効率的なプログラミングが可能になります。Pythonは、適切なツール', 'learning_step': '\n\n1. Pythonの基本コマンドを学ぶ。\n2. ライブラリを使用して特定のタスクを実行する方法を学ぶ。\n3. Pythonを使用して自分自身のプログラムを開発する方法を学ぶ。'}


In [8]:
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

memory.save_context(
    {"input": "AIとは何？"},
    {"output": "AIとは、人工知能のことです。"},
)
print(memory.load_memory_variables({}))

ValidationError: 1 validation error for HumanMessage
content
  str type expected (type=type_error.str)

In [5]:
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)

memory.save_context(
    {"input": "AIとは何？"},
    {"output": "AIとは、人工知能のことです。"},
)
print(memory.load_memory_variables({}))

{'history': [HumanMessage(content='AIとは何？', additional_kwargs={}, example=False), AIMessage(content='AIとは、人工知能のことです。', additional_kwargs={}, example=False)]}


In [9]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

llm = OpenAI(model_name="text-davinci-003")
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

conversation("AIとは何？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: AIとは何？
AI:

> Finished chain.


{'input': 'AIとは何？',
 'history': '',
 'response': ' AIとは人工知能の略です。人工知能とは、コンピューターや機械に人間のような思考や行動をさせる技術のことです。コンピューターや機械が、データや知識を元に複雑なタスクを自動的に実行することができるようになります。'}

In [10]:
conversation("より詳しく教えて。")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: AIとは何？
AI:  AIとは人工知能の略です。人工知能とは、コンピューターや機械に人間のような思考や行動をさせる技術のことです。コンピューターや機械が、データや知識を元に複雑なタスクを自動的に実行することができるようになります。
Human: より詳しく教えて。
AI:

> Finished chain.


{'input': 'より詳しく教えて。',
 'history': 'Human: AIとは何？\nAI:  AIとは人工知能の略です。人工知能とは、コンピューターや機械に人間のような思考や行動をさせる技術のことです。コンピューターや機械が、データや知識を元に複雑なタスクを自動的に実行することができるようになります。',
 'response': ' 人工知能は、機械学習、自然言語処理、画像認識などのさまざまな分野で使われています。機械学習では、コンピューターがデータを分析して、自動的に新しい発見を行うことができます。自然言語処理では、コンピューターが文章を理解して、自動的に応答を行うことができます。画像認識では、コンピューターが画像を認識して、特定の物体を見つけ出すことができます。'}

In [11]:
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain, PromptTemplate

template = """あなたは人間と話すチャットボットです。

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"],
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(
    llm=OpenAI(model_name="text-davinci-003"),
    prompt=prompt,
    verbose=True,
    memory=memory,
)

llm_chain.predict(human_input="AIとは何？")



> Entering new LLMChain chain...
Prompt after formatting:
あなたは人間と話すチャットボットです。


Human: AIとは何？
Chatbot:

> Finished chain.


' AIとは、人工知能（Artificial Intelligence）の略です。AIは、コンピューターを使って人間のように学習し、思考する能力を獲得するプログラミング技術です。'

In [13]:
llm_chain.predict(human_input="より詳しく教えて。")



> Entering new LLMChain chain...
Prompt after formatting:
あなたは人間と話すチャットボットです。

Human: AIとは何？
AI:  AIとは、人工知能（Artificial Intelligence）の略です。AIは、コンピューターを使って人間のように学習し、思考する能力を獲得するプログラミング技術です。
Human: より詳しく教えて。
Chatbot:

> Finished chain.


' AIは、コンピューターを使って機械学習を利用して、複雑なタスクを自動的に実行するためのプログラミング技術です。AIを使用することで、コンピューターが自動的に人間の行動を模倣したり、複雑なプロセスを自動的に実行したりできます。また、AIは、コンピューターが大量のデータからパターンを分析し、以前の知識を活用して予測と推論を行う能力を高めるためにも使用されます。'